In [ ]:
!pip install -q transformers
!pip install mlconjug3

     |████████████████████████████████| 2.3MB 6.9MB/s 
     |████████████████████████████████| 901kB 44.5MB/s 
     |████████████████████████████████| 3.3MB 48.0MB/s 
     |████████████████████████████████| 35.3MB 120kB/s 
     |████████████████████████████████| 22.3MB 1.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline

from collections import Counter

import mlconjug3

import json

In [ ]:
nlp_fill = pipeline('fill-mask',model="neuralmind/bert-base-portuguese-cased", use_fast=True, top_k=10)
# nlp_fill("Eu vou [MASK] uma palavra")


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
conjugator = mlconjug3.Conjugator(language='pt')

In [ ]:
def load_pos_dict(path):
  # abaixe-vos,abaixar.V+PRO:Y3s
  # abaixo-a,abaixar.V+PRO:P1s
  # abaixo,abaixar.V:P1s
  # abaixo,abaixo.ADV
  puncts = "._*—,;!?:-–()[]{}'«»•\""
  pos_dict = {}
  with open(path) as inputfile:
    for ln in inputfile:
      w, pos = ln.strip().split(".")
      w = w.split(",")[0]
      # pos = pos.split(":")[0].split("+")[0]
      if w not in pos_dict:
        pos_dict[w] = set()
      pos_dict[w].add(pos)
  print(len(pos_dict), "types read")
  for p in puncts:
    pos_dict[p] = set(["punct"])  
  return pos_dict


pos_dict = load_pos_dict("/content/drive/MyDrive/Aline et al/Delaf2015v04.dic") # http://www.nilc.icmc.usp.br/nilc/projects/unitex-pb/web/dicionarios.html
                                           # www.nilc.icmc.usp.br/nilc/projects/unitex-pb/web/files/grafos_inflecion_DELAF_PB_v2.zip


6572958 types read


In [ ]:
def process(seeds, sentences, use_cls, use_sep, nlp_fill, pos_dict, task, verbose = True):
  outputs = {}
  results_scores_ret = {}
  results_pos_ret = {}
  results_ood_ret = {}
  for value in seeds:
    results_scores = {}
    results_pos = {}
    results_ood = set()
    results_scores_ret = {value:results_scores}
    results_pos_ret = {value:results_pos}
    results_ood_ret = {value:results_ood}
    for seed in seeds[value]:
      if verbose:
        print(value,seed)
      if True: #try:
        results_scores[seed] = []
        results_pos[seed] = Counter()
        for sent in sentences:
          sent = sent.replace("<SEED>", seed)
          if use_cls:
            sent = "[CLS] " + sent
          if use_sep:
            sent = sent.strip() + " [SEP]"
          if verbose:
            print(sent)
          candidates = nlp_fill(sent)
          key = task + "\t" + seed+"\t"+ sent
          output = []
          for cand in candidates:
            results_scores[seed].append((cand["token_str"], cand["score"]))
            all_pos = pos_dict[cand["token_str"].lower()] if cand["token_str"].lower() in pos_dict else ["bert_tk"] if cand["token_str"].startswith("##") else ["OOD"] #out of dictionary
            output.append( (";".join(all_pos), cand["token_str"], cand["score"]) )
            
            for pos in all_pos:
              results_pos[seed][pos] += 1
              if pos == "OOD":
                results_ood.add(cand["token_str"])
            if verbose:
              print("\t", cand["token_str"], cand["score"], all_pos)
          outputs[key] = output
      # except:
      #   print("bad run at seed: %s in value: %s" %(value, seed))
  return results_scores_ret, results_pos_ret, results_ood_ret, outputs


#templates

In [ ]:
TASKS = {}

##templates 1 - verbos sem sujeito.txt

In [ ]:
base_verbs = ["chover", "trovejar", "ventar", "nevar", "relampejar"]
tenses = [ ('Indicativo', 'Indicativo presente'), ('Indicativo','Indicativo pretérito imperfeito'), ('Indicativo','Indicativo Pretérito Mais que Perfeito Simples'), 
          ('Indicativo','Indicativo pretérito perfeito simples'), ('Indicativo','Indicativo Futuro do Presente Simples'), ('Condicional','Condicional Futuro do Pretérito Simples')]
seeds = {}
for mood, tense in tenses:
   seeds[mood +"/"+ tense] = set()
for mood, tense in tenses:
  for verb in base_verbs:
    seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
    seed = seed[0].upper() + seed[1:]
    seeds[mood +"/"+ tense].add(seed)
for mood, tense in tenses:
   seeds[mood +"/"+ tense] = list(seeds[mood +"/"+ tense])
sentences = ["[MASK] <SEED> toda a manhã e de tarde ficou meio assim, aí a gente veio igual, mas já achando que vocês não tavam aqui.",
             "[MASK] <SEED> na cidade durante 12 horas seguidas, de sábado a domingo.",
             "[MASK] <SEED> à tarde."]
for index, sent in enumerate(sentences):
  template1 = (seeds, [sent])
  TASKS["verbo_sem_sujeito_1_" + str(index)] = template1

In [ ]:
base_verbs = ["fazer", "haver"]
tenses = [ ('Indicativo', 'Indicativo presente'), ('Indicativo','Indicativo pretérito imperfeito'), ('Indicativo','Indicativo Pretérito Mais que Perfeito Simples'), 
          ('Indicativo','Indicativo pretérito perfeito simples'), ('Indicativo','Indicativo Futuro do Presente Simples'), ('Condicional','Condicional Futuro do Pretérito Simples')]
seeds = {}
for mood, tense in tenses:
   seeds[mood +"/"+ tense] = set()
for mood, tense in tenses:
  for verb in base_verbs:
    seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
    seed = seed[0].upper() + seed[1:]
    seeds[mood +"/"+ tense].add(seed)
for mood, tense in tenses:
   seeds[mood +"/"+ tense] = list(seeds[mood +"/"+ tense])
sentences = ["[MASK] <SEED> anos que coleciono essas pérolas com as quais os professores de literatura pervertem seus alunos.",
             "[MASK] <SEED> anos que não me sensibilizo tanto numa exposição.",
             "[MASK] <SEED> anos que isso não acontecia."]
for index, sent in enumerate(sentences):
  template2 = (seeds, [sent])
  TASKS["verbo_sem_sujeito_2_" + str(index)] = template2

##templates 2 - concordância de sujeito.txt

In [ ]:
tenses = [ ('Indicativo', 'Indicativo presente'), ('Indicativo','Indicativo pretérito imperfeito'), ('Indicativo','Indicativo Pretérito Mais que Perfeito Simples'), 
          ('Indicativo','Indicativo pretérito perfeito simples'), ('Indicativo','Indicativo Futuro do Presente Simples'), ('Condicional','Condicional Futuro do Pretérito Simples')]
seeds = {}
for mood, tense in tenses:
   seeds[tense+"_sg" ] = set()
   seeds[tense+"_pl" ] = set()
   seeds[tense+"_2pl" ] = set()
for mood, tense in tenses:
  verb = "adotar"
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1p']
  seeds[tense+"_pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2p']
  seeds[tense+"_2pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3p']
  seeds[tense+"_pl"].add(seed)
for mood, tense in tenses:
   seeds[tense+"_sg" ] = list(seeds[tense+"_sg" ])
   seeds[tense+"_pl" ] = list(seeds[tense+"_pl" ])
   seeds[tense+"_2pl" ] = list(seeds[tense+"_2pl" ])
sentences = ["Deputados, no decurso destes anos em que a Constituição Cidadã está em vigor, podemos observar o acerto de muitas das medidas que [MASK] <SEED>."]

template3 = (seeds, sentences)
TASKS["concordancia_de_sujeito_1_"+verb] = template3

tenses = [ ('Indicativo', 'Indicativo presente'), ('Indicativo','Indicativo pretérito imperfeito'), ('Indicativo','Indicativo Pretérito Mais que Perfeito Simples'), 
          ('Indicativo','Indicativo pretérito perfeito simples'), ('Indicativo','Indicativo Futuro do Presente Simples'), ('Condicional','Condicional Futuro do Pretérito Simples')]
seeds = {}
for mood, tense in tenses:
   seeds[tense+"_sg" ] = set()
   seeds[tense+"_pl" ] = set()
   seeds[tense+"_2pl" ] = set()
for mood, tense in tenses:
  verb = "afirmar"
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1p']
  seeds[tense+"_pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2p']
  seeds[tense+"_2pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3p']
  seeds[tense+"_pl"].add(seed)
for mood, tense in tenses:
   seeds[tense+"_sg" ] = list(seeds[tense+"_sg" ])
   seeds[tense+"_pl" ] = list(seeds[tense+"_pl" ])
   seeds[tense+"_2pl" ] = list(seeds[tense+"_2pl" ])
sentences = ["[MASK] também <SEED> que, por ter problemas cardíacos, a seringa continha um \"remédio para o coração\"."]

template3 = (seeds, sentences)
TASKS["concordancia_de_sujeito_1_"+verb] = template3

tenses = [ ('Indicativo', 'Indicativo presente'), ('Indicativo','Indicativo pretérito imperfeito'), ('Indicativo','Indicativo Pretérito Mais que Perfeito Simples'), 
          ('Indicativo','Indicativo pretérito perfeito simples'), ('Indicativo','Indicativo Futuro do Presente Simples'), ('Condicional','Condicional Futuro do Pretérito Simples')]
seeds = {}
for mood, tense in tenses:
   seeds[tense+"_sg" ] = set()
   seeds[tense+"_pl" ] = set()
   seeds[tense+"_2pl" ] = set()
for mood, tense in tenses:
  verb = "ser"
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1p']
  seeds[tense+"_pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2p']
  seeds[tense+"_2pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3p']
  seeds[tense+"_pl"].add(seed)
for mood, tense in tenses:
   seeds[tense+"_sg" ] = list(seeds[tense+"_sg" ])
   seeds[tense+"_pl" ] = list(seeds[tense+"_pl" ])
   seeds[tense+"_2pl" ] = list(seeds[tense+"_2pl" ])
sentences = ["[MASK] <SEED> produto de uma época."]

template3 = (seeds, sentences)
TASKS["concordancia_de_sujeito_1_"+verb] = template3

tenses = [ ('Indicativo', 'Indicativo presente'), ('Indicativo','Indicativo pretérito imperfeito'), ('Indicativo','Indicativo Pretérito Mais que Perfeito Simples'), 
          ('Indicativo','Indicativo pretérito perfeito simples'), ('Indicativo','Indicativo Futuro do Presente Simples'), ('Condicional','Condicional Futuro do Pretérito Simples')]
seeds = {}
for mood, tense in tenses:
   seeds[tense+"_sg" ] = set()
   seeds[tense+"_pl" ] = set()
   seeds[tense+"_2pl" ] = set()
for mood, tense in tenses:
  verb = "comentar"
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1p']
  seeds[tense+"_pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2p']
  seeds[tense+"_2pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3p']
  seeds[tense+"_pl"].add(seed)
for mood, tense in tenses:
   seeds[tense+"_sg" ] = list(seeds[tense+"_sg" ])
   seeds[tense+"_pl" ] = list(seeds[tense+"_pl" ])
   seeds[tense+"_2pl" ] = list(seeds[tense+"_2pl" ])
sentences = ["Neste contexto, como [MASK] <SEED>, o tempo de compreender se evapora."]

template3 = (seeds, sentences)
TASKS["concordancia_de_sujeito_1_"+verb] = template3

tenses = [ ('Indicativo', 'Indicativo presente'), ('Indicativo','Indicativo pretérito imperfeito'), ('Indicativo','Indicativo Pretérito Mais que Perfeito Simples'), 
          ('Indicativo','Indicativo pretérito perfeito simples'), ('Indicativo','Indicativo Futuro do Presente Simples'), ('Condicional','Condicional Futuro do Pretérito Simples')]
seeds = {}
for mood, tense in tenses:
   seeds[tense+"_sg" ] = set()
   seeds[tense+"_pl" ] = set()
   seeds[tense+"_2pl" ] = set()
for mood, tense in tenses:
  verb = "justificar"
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1p']
  seeds[tense+"_pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2p']
  seeds[tense+"_2pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3p']
  seeds[tense+"_pl"].add(seed)
for mood, tense in tenses:
   seeds[tense+"_sg" ] = list(seeds[tense+"_sg" ])
   seeds[tense+"_pl" ] = list(seeds[tense+"_pl" ])
   seeds[tense+"_2pl" ] = list(seeds[tense+"_2pl" ])
sentences = ["[MASK] <SEED> de duas maneiras sua forma de pensar."]

template3 = (seeds, sentences)
TASKS["concordancia_de_sujeito_1_"+verb] = template3

tenses = [ ('Indicativo', 'Indicativo presente'), ('Indicativo','Indicativo pretérito imperfeito'), ('Indicativo','Indicativo Pretérito Mais que Perfeito Simples'), 
          ('Indicativo','Indicativo pretérito perfeito simples'), ('Indicativo','Indicativo Futuro do Presente Simples'), ('Condicional','Condicional Futuro do Pretérito Simples')]
seeds = {}
for mood, tense in tenses:
   seeds[tense+"_sg" ] = set()
   seeds[tense+"_pl" ] = set()
   seeds[tense+"_2pl" ] = set()
for mood, tense in tenses:
  verb = "estar"
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1p']
  seeds[tense+"_pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2p']
  seeds[tense+"_2pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3p']
  seeds[tense+"_pl"].add(seed)
for mood, tense in tenses:
   seeds[tense+"_sg" ] = list(seeds[tense+"_sg" ])
   seeds[tense+"_pl" ] = list(seeds[tense+"_pl" ])
   seeds[tense+"_2pl" ] = list(seeds[tense+"_2pl" ])
sentences = ["[MASK] <SEED> na empresa desde 1984."]

template3 = (seeds, sentences)
TASKS["concordancia_de_sujeito_1_"+verb] = template3




In [ ]:
tenses = [ ('Indicativo', 'Indicativo presente'), ('Indicativo','Indicativo pretérito imperfeito'), ('Indicativo','Indicativo Pretérito Mais que Perfeito Simples'), 
          ('Indicativo','Indicativo pretérito perfeito simples'), ('Indicativo','Indicativo Futuro do Presente Simples'), ('Condicional','Condicional Futuro do Pretérito Simples')]
seeds = {}
for mood, tense in tenses:
   seeds[tense+"_sg" ] = set()
   seeds[tense+"_pl" ] = set()
   seeds[tense+"_2pl" ] = set()
for mood, tense in tenses:
  verb = "contribuir"
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1p']
  seeds[tense+"_pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2p']
  seeds[tense+"_2pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3p']
  seeds[tense+"_pl"].add(seed)
for mood, tense in tenses:
   seeds[tense+"_sg" ] = list(seeds[tense+"_sg" ])
   seeds[tense+"_pl" ] = list(seeds[tense+"_pl" ])
   seeds[tense+"_2pl" ] = list(seeds[tense+"_2pl" ])
sentences = ["[MASK] não <SEED> para modificar a situação de angústia da população, ratificada com o resultado das últimas eleições, quando o povo teve a oportunidade de expressar sua revolta."]

template3 = (seeds, sentences)
TASKS["concordancia_de_sujeito_2_"+verb] = template3

seeds = {}
for mood, tense in tenses:
   seeds[tense+"_sg" ] = set()
   seeds[tense+"_pl" ] = set()
   seeds[tense+"_2pl" ] = set()
for mood, tense in tenses:
  verb = "falar"
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1p']
  seeds[tense+"_pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2p']
  seeds[tense+"_2pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3p']
  seeds[tense+"_pl"].add(seed)
for mood, tense in tenses:
   seeds[tense+"_sg" ] = list(seeds[tense+"_sg" ])
   seeds[tense+"_pl" ] = list(seeds[tense+"_pl" ])
   seeds[tense+"_2pl" ] = list(seeds[tense+"_2pl" ])
sentences = ["Mas [MASK] <SEED> que elas são ruins, então não tem nada que você gostaria de mudar?"]

template3 = (seeds, sentences)
TASKS["concordancia_de_sujeito_2_"+verb] = template3

seeds = {}
for mood, tense in [('Indicativo','Indicativo pretérito perfeito simples')]:
  #  seeds[tense+"_sg" ] = []
   seeds[tense+"_pl" ] = set()
  #  seeds[tense+"_2pl" ] = []
for mood, tense in [('Indicativo','Indicativo pretérito perfeito simples')]:
  verb = "fazer"
  # seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1s']
  # seeds[tense+"_sg"].append(seed)
  # seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2s']
  # seeds[tense+"_sg"].append(seed)
  # seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
  # seeds[tense+"_sg"].append(seed)
  # seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1p']
  # seeds[tense+"_pl"].append(seed)
  # seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2p']
  # seeds[tense+"_2pl"].append(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3p']
  seeds[tense+"_pl"].add(seed)
for mood, tense in [('Indicativo','Indicativo pretérito perfeito simples')]:
   seeds[tense+"_pl" ] = list(seeds[tense+"_pl" ])
sentences = ["Na maternidade [MASK] <SEED> de tudo para aliviar a dor, me colocaram na bola, no cavalinho, fizeram de tudo para ser parto normal, pois até então, o bebê estava encaixado."]

template3 = (seeds, sentences)
TASKS["concordancia_de_sujeito_2_"+verb] = template3

seeds = {}
for mood, tense in tenses:
   seeds[tense+"_sg" ] = set()
   seeds[tense+"_pl" ] = set()
   seeds[tense+"_2pl" ] = set()
for mood, tense in tenses:
  verb = "carregar"
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3s']
  seeds[tense+"_sg"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['1p']
  seeds[tense+"_pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['2p']
  seeds[tense+"_2pl"].add(seed)
  seed = conjugator.conjugate(verb).conjug_info[mood][tense]['3p']
  seeds[tense+"_pl"].add(seed)
for mood, tense in tenses:
   seeds[tense+"_sg" ] = list(seeds[tense+"_sg" ])
   seeds[tense+"_pl" ] = list(seeds[tense+"_pl" ])
   seeds[tense+"_2pl" ] = list(seeds[tense+"_2pl" ])
sentences = ["[MASK] <SEED> 7.280 papelotes de cocaína, avaliados em cerca de R$ 36.400."]

template3 = (seeds, sentences)
TASKS["concordancia_de_sujeito_2_"+verb] = template3

##templates 3 - concordância verbal.txt

In [ ]:
all_sentences = ["Ontem, <ART> [MASK] vistoriou construções de controle de enchentes e de abastecimento de água na região do Alto Tietê, sobrevoando parte dos 27 quilômetros do rio Cabuçu de Cima e das barragens dos rios Biritiba, na divisa dos municípios de Biritiba Mirim e Mogi das Cruzes, e Paraitinga, em Salesópolis.", 
             "Ontem <ART> [MASK] completou 62 dias, tornando o sequestro o mais longo da América Latina envolvendo embaixadas.", 
             "Ontem <ART> [MASK] encontrou Maria Aparecida para receber o dinheiro.", 
             "Ontem, <ART> [MASK] comemorou a decisão da Justiça e disse que ela comprova a improcedência da ação movida pela AGU."]
sentences = ["Ontem, <SEED> [MASK] construções de controle de enchentes e de abastecimento de água na região do Alto Tietê, sobrevoando parte dos 27 quilômetros do rio Cabuçu de Cima e das barragens dos rios Biritiba, na divisa dos municípios de Biritiba Mirim e Mogi das Cruzes, e Paraitinga, em Salesópolis.", 
             "Ontem <SEED> [MASK] 62 dias, tornando o sequestro o mais longo da América Latina envolvendo embaixadas.", 
             "Ontem <SEED> [MASK] Maria Aparecida para receber o dinheiro.", 
             "Ontem, <SEED> [MASK] a decisão da Justiça e disse que ela comprova a improcedência da ação movida pela AGU." ]

pattern = [("sg_m","o"), ("sg_f","a"), ("pl_m", "os"), ("pl_f", "as")]

for index, (sent, real_sent) in enumerate(zip(all_sentences,sentences)):
  seeds = {"sg":set(),"pl":set()}
  for art in pattern:
    # print(sent)
    words = nlp_fill(sent.replace("<ART>", art[1]))
    for word in words:
      # print("\t",word["token_str"], word["score"])
      if word["score"] < 0.001:
        continue
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      seeds[art[0].split("_")[0]].add(art[1] + " " + word["token_str"].lower())
  seeds["sg"] = list(seeds["sg"])
  seeds["pl"] = list(seeds["pl"])
  seeds["pron_sg"] = ["eu", "tu", "ele", "ela"]
  seeds["pron_pl"] = ["nós", "vós", "eles", "elas"]
  TASKS["concordância verbal_1_" + str(index)] = (seeds, [real_sent])
# seeds = {"art_sg+noun_sg":["o governador", "a governadora", "a crise", "o senador", "a senadora", "o prazo"],
#          "art_pl+noun_pl":["os governadores", "as governadoras", "as crises", "os senadores", "as senadoras"],
#          "pron_sg":["eu", "tu", "ele", "ela"],
#          "pron_pl":["nós", "vós", "eles", "elas"]}



# for seed in seeds:
#   print(seed, seeds[seed])

In [ ]:
all_sentences = ["Amanhã, <ART> [MASK] estréia no torneio de simples do Torneio de Maurier, em Montreal (Canadá), abrindo uma série de competições em quadras sintéticas.",
             "Amanhã, com certeza, <ART> [MASK] terminarei as atividades relativas a unidade 02.",
             "Amanhã também vence <ART> [MASK] para o pagamento da primeira cota (ou única).",
             "Amanhã <ART> [MASK] cozinho!"]
sentences = ["Amanhã, <SEED> [MASK] no torneio de simples do Torneio de Maurier, em Montreal (Canadá), abrindo uma série de competições em quadras sintéticas.", 
             "Amanhã, com certeza, <SEED> [MASK] as atividades relativas a unidade 02.", 
             "Amanhã também [MASK] <SEED> para o pagamento da primeira cota (ou única).",
             "Amanhã <SEED> [MASK]!"]

pattern = [("sg_m","o"), ("sg_f","a"), ("pl_m", "os"), ("pl_f", "as")]

for index, (sent, real_sent) in enumerate(zip(all_sentences,sentences)):
  seeds = {"sg":set(),"pl":set()}
  for art in pattern:
    # print(sent)
    words = nlp_fill(sent.replace("<ART>", art[1]))
    for word in words:
      # print("\t",word["token_str"], word["score"])
      if word["score"] < 0.001:
        continue
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      seeds[art[0].split("_")[0]].add(art[1] + " " + word["token_str"].lower())
  seeds["sg"] = list(seeds["sg"])
  seeds["pl"] = list(seeds["pl"])
  seeds["pron_sg"] = ["eu", "tu", "ele", "ela"]
  seeds["pron_pl"] = ["nós", "vós", "eles", "elas"]

  TASKS["concordância verbal_2_" + str(index)] = (seeds, [real_sent])

# for seed in seeds:
#   print(seed, seeds[seed])

In [ ]:
seeds = {"seeds":[ "queriam", "quereriam", "poderiam", "teriam de", "precisariam", "iriam", "gostariam de"]}
sentences = ["Schiller (1949/1978) ofereceu objetos a chimpanzés de cativeiro e depois submeteu-os a testes em que <SEED> [MASK] tais objetos para solucionar tarefas."]
TASKS["concordância verbal_3a"] = (seeds, sentences)

seeds = {"seeds":["queremos", "poderíamos", "temos de", "teríamos de", "precisamos", "precisaríamos", "vamos", "iremos", "iríamos", "devemos", "deveríamos", "gostaríamos de"]}
sentences = ["Os pronomes aparecem nas sentenças em lugar de substantivos, então <SEED> [MASK] duas sentenças através desse substantivo referido pelo pronome."]
TASKS["concordância verbal_3b"] = (seeds, sentences)

seeds = {"seeds":["queria", "poderia", "teria de", "precisaria", "deveria", "gostaria de"]}
sentences = ["O aluno Hélio gostou do convite e resolveu imediatamente que não <SEED> [MASK]."]
TASKS["concordância verbal_3c"] = (seeds, sentences)

seeds = {"seeds":["queremos", "poderíamos", "temos de", "teríamos de", "precisamos", "precisaríamos", "vamos", "iremos", "iríamos", "devemos", "deveríamos", "gostaríamos de"]}
sentences = ["Diz o que nós não <SEED> [MASK]."]
TASKS["concordância verbal_3d"] = (seeds, sentences)


In [ ]:
sentencesSeed = ["No entanto, no decorrer do trabalho percebi que além de pesquisar sobre a importância de incluir estudos sobre o desenvolvimento humano na FC eu [MASK] lidando com algo que tinha abrangência e impacto maiores do que previa.", 
                 "Com a ajuda de doações, a família das gêmeas, que não dispõe de recursos financeiros, pois o pai, Francisco Canindé da Silva, tem uma renda mensal de apenas 530 reais, [MASK] recebendo ajuda de pessoas e é assistida também pelos médicos que participaram da equipe que efetuou a separação das siamesas.",
                 "Consultou técnicos da companhia urbanizadora que já [MASK] trabalhando com o loteamento da Mitra, \"se feito o levantamento, o negócio fosse exeqüível, lhes dariam andamento\".", 
                 "Eu [MASK] retomando uma certa coisa que eu já tinha vivido, como se já tivesse vivido antes.", 
                 "Braga disse que os familiares do pastor [MASK] negociando a redução desse valor.", 
                 "[MASK] tentando defender o Sr."]

sentences = ["No entanto, no decorrer do trabalho percebi que além de pesquisar sobre a importância de incluir estudos sobre o desenvolvimento humano na FC eu <SEED> [MASK] com algo que tinha abrangência e impacto maiores do que previa.",
            "Com a ajuda de doações, a família das gêmeas, que não dispõe de recursos financeiros, pois o pai, Francisco Canindé da Silva, tem uma renda mensal de apenas 530 reais, <SEED> [MASK] ajuda de pessoas e é assistida também pelos médicos que participaram da equipe que efetuou a separação das siamesas.",
            "Consultou técnicos da companhia urbanizadora que já <SEED> [MASK] com o loteamento da Mitra, \"se feito o levantamento, o negócio fosse exeqüível, lhes dariam andamento\".",
            "Eu <SEED> [MASK] uma certa coisa que eu já tinha vivido, como se já tivesse vivido antes.",
            "Braga disse que os familiares do pastor <SEED> [MASK] a redução desse valor.",
            "<SEED> [MASK] defender o Sr." ]

seed_sent = []
for sent in sentencesSeed:
  words = nlp_fill(sent)
  seeds = []
  for word in words:
    if word["score"] < 0.001:
      continue
    if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
      continue
    seeds.append(word["token_str"].lower())
  seed_sent.append(seeds)


for index, (sent, seed_lst) in enumerate(zip(sentences, seed_sent)):
  seeds = {}
  seeds["case_" + str(index)] = seed_lst
  TASKS["concordância verbal_4_" + str(index)] = (seeds, [sent])




In [ ]:
seeds = {}
seeds["pron_eu_ele_vc"] = ["eu", "ele", "ela", "você"]
seeds["pron_tu"] = ["tu"]
seeds["pron_nos"] = ["nós"]
seeds["pron_vos"] = ["vós"]
seeds["pron_eles_vcs"] = ["eles", "elas", "vocês"]

all_sentences = ["E, quando quer que <SEED> [MASK] um suposto intelectual falar em \"A verdade\", \"O caminho\" e demais arbitrariedades, saia correndo.",
             "Por esse lado, é possível que <SEED> [MASK] mais pressões.",
             "E aconselhável que <SEED> [MASK], com as crianças, lembretes que devem ser afixados em um canto da sala, para ajudar nas composições individuais, a partir da 3.",
             "Jatobá - O ideal é que <SEED> [MASK] um desemprego conjuntural, decorrente da retração da atividade econômica, nulo.",
             "Espera-se que <SEED> [MASK] as notas introdutórias relativas a uma determinada unidade antes de cada sessão do curso."]

for index, sent in enumerate(all_sentences):
  TASKS["concordância verbal_5_" + str(index)] = (seeds, [sent])

for seed in seeds:
  print(seed, seeds[seed])


pron_eu_ele_vc ['eu', 'ele', 'ela', 'você']
pron_tu ['tu']
pron_nos ['nós']
pron_vos ['vós']
pron_eles_vcs ['eles', 'elas', 'vocês']


In [ ]:
seeds = {}

seeds["pron_eu_ele_vc"] = ["eu", "ele", "ela", "você"]
seeds["pron_tu"] = ["tu"]
seeds["pron_nos"] = ["nós"]
seeds["pron_vos"] = ["vós"]
seeds["pron_eles_vcs"] = ["eles", "elas", "vocês"]

all_sentences = ["Como se <SEED> [MASK] uma marca de ferro quente em sua garganta abaixo.",
             "Se <SEED> [MASK] um segundo diante de cada uma das 30 mil peças, levaria de oito horas e não veria nada.",
             "Se <SEED> [MASK] esse jornal que está lendo agora no mar, ele levaria seis semanas para se dissolver e sumir.",
             "E, se <SEED> [MASK] um Rei tão bondoso e brincalhão, nós temos certeza de você estaria sorrindo também!",
             "Seria muito bom se <SEED> [MASK] participar!", 
             "E como se <SEED> [MASK] em outro mundo."]
for index, sent in enumerate(all_sentences):
  TASKS["concordância verbal_6_" + str(index)] = (seeds, [sent])

for seed in seeds:
  print(seed, seeds[seed])


pron_eu_ele_vc ['eu', 'ele', 'ela', 'você']
pron_tu ['tu']
pron_nos ['nós']
pron_vos ['vós']
pron_eles_vcs ['eles', 'elas', 'vocês']


In [ ]:
sentencesSeed = ["Se <ART> [MASK] disparar cem flechas seguidas, o que acontecerá com o seu arco?",
                 "Se <ART> [MASK] reservar um tempo regular para praticar o essencial da oração, discutido tão brevemente acima, tornar-se-á, com certeza, mais consciente da presença de Deus em sua vida.",
                 "Se <ART> [MASK] fechar os olhos , só poderá explorar o interior de uma casca de noz com um dedo, não podendo fazê-lo com o seu punho ou a cabeça.",
                 "Se <ART> [MASK] preferir utilizar o SQL Server, garanta que o driver ODBC está instalado.",
                 "O seu carro, se <ART> [MASK] não tiver vai ser seu A8:"]

sentences = ["Se <SEED> [MASK] cem flechas seguidas, o que acontecerá com o seu arco?",
             "Se <SEED> [MASK] um tempo regular para praticar o essencial da oração, discutido tão brevemente acima, tornar-se-á, com certeza, mais consciente da presença de Deus em sua vida.",
             "Se <SEED> [MASK] os olhos , só poderá explorar o interior de uma casca de noz com um dedo, não podendo fazê-lo com o seu punho ou a cabeça.",
             "Se <SEED> [MASK] utilizar o SQL Server, garanta que o driver ODBC está instalado.",
             "O seu carro, se <SEED> não [MASK] vai ser seu A8:"]

seed_sent = []
pattern = [("sg_m","o"), ("sg_f","a"), ("pl_m", "os"), ("pl_f", "as")]
for sent in sentencesSeed:
  seeds = []
  for art in pattern:
  
    words = nlp_fill(sent.replace("<ART>", art[1]))
    
    for word in words:
      if word["score"] < 0.001:
        continue
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      seeds.append(art[1] + " " + word["token_str"].lower())
  seed_sent.append(seeds)


for index, (sent, seed_lst) in enumerate(zip(sentences, seed_sent)):
  seeds = {}
  seeds["case_" + str(index)] = seed_lst
  TASKS["concordância verbal_7_" + str(index)] = (seeds, [sent])



##templates 4 - concordância nominal.txt

In [ ]:
patterns = {"m_sg" : "Foi à luz desse critério, que a ilustre Relatora do Parecer nº2121/78 entendeu filiada ao [MASK] estadual de ensino a Fundação Educacional Presidente Castelo Branco, visto que, segundo os dados disponíveis, sua fonte principal de receita eram as subvenções orçamentárias do Município e ao Prefeito cabia nomear-lhe o Presidente e o Diretor Executivo, atribuindo-se à Câmara de Vereadores de Colatina determinar o destino de seu patrimônio, em caso de dissolução.", 
            "m_pl" : "Foi à luz desse critério, que a ilustre Relatora do Parecer nº2121/78 entendeu filiada aos [MASK] estaduais de ensino a Fundação Educacional Presidente Castelo Branco, visto que, segundo os dados disponíveis, sua fonte principal de receita eram as subvenções orçamentárias do Município e ao Prefeito cabia nomear-lhe o Presidente e o Diretor Executivo, atribuindo-se à Câmara de Vereadores de Colatina determinar o destino de seu patrimônio, em caso de dissolução.", 
            "f_sg" : "Foi à luz desse critério, que a ilustre Relatora do Parecer nº2121/78 entendeu filiada à [MASK] estadual de ensino a Fundação Educacional Presidente Castelo Branco, visto que, segundo os dados disponíveis, sua fonte principal de receita eram as subvenções orçamentárias do Município e ao Prefeito cabia nomear-lhe o Presidente e o Diretor Executivo, atribuindo-se à Câmara de Vereadores de Colatina determinar o destino de seu patrimônio, em caso de dissolução.", 
            "f_pl" : "Foi à luz desse critério, que a ilustre Relatora do Parecer nº2121/78 entendeu filiada às [MASK] estaduais de ensino a Fundação Educacional Presidente Castelo Branco, visto que, segundo os dados disponíveis, sua fonte principal de receita eram as subvenções orçamentárias do Município e ao Prefeito cabia nomear-lhe o Presidente e o Diretor Executivo, atribuindo-se à Câmara de Vereadores de Colatina determinar o destino de seu patrimônio, em caso de dissolução."}
seeds = {}
for pattern in patterns:
  seeds[pattern] = []

for pattern in patterns:
    sent = patterns[pattern]
    words = nlp_fill(sent)
    art = sent.split("[MASK]")[0].strip().split()[-1]
    # print(sent)
    for word in words:
      # print("\t",word["token_str"], word["score"])
      if word["score"] < 0.001:
        continue
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      word = art + " " + word["token_str"].lower()
      if word not in seeds[pattern]:
        seeds[pattern].append(word)
sentences = ["Foi à luz desse critério, que a ilustre Relatora do Parecer nº2121/78 entendeu filiada <SEED> [MASK] de ensino a Fundação Educacional Presidente Castelo Branco, visto que, segundo os dados disponíveis, sua fonte principal de receita eram as subvenções orçamentárias do Município e ao Prefeito cabia nomear-lhe o Presidente e o Diretor Executivo, atribuindo-se à Câmara de Vereadores de Colatina determinar o destino de seu patrimônio, em caso de dissolução."]

TASKS["concordância nominal_1a"] = (seeds, sentences)


patterns = {"m_sg" : "Em [MASK] absoluto, o número total de professores contratados entre 1972 e 1978 aumentou de 58.278 para 93.929.", 
            "m_pl" : "Em [MASK] absolutos, o número total de professores contratados entre 1972 e 1978 aumentou de 58.278 para 93.929.", 
            "f_sg" : "Em [MASK] absoluta, o número total de professores contratados entre 1972 e 1978 aumentou de 58.278 para 93.929.", 
            "f_pl" : "Em [MASK] absolutas, o número total de professores contratados entre 1972 e 1978 aumentou de 58.278 para 93.929."}
seeds = {}
for pattern in patterns:
  seeds[pattern] = []

for pattern in patterns:
    sent = patterns[pattern]
    words = nlp_fill(sent)
    art = sent.split("[MASK]")[0].strip().split()[-1]
    # print(sent)
    for word in words:
      if word["score"] < 0.001:
        continue
      # print("\t",word["token_str"], word["score"])
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      word = art + " " + word["token_str"].lower()
      if word not in seeds[pattern]:
        seeds[pattern].append(word)
sentences = ["<SEED> [MASK] o número total de professores contratados entre 1972 e 1978 aumentou de 58.278 para 93.929."]

TASKS["concordância nominal_1b"] = (seeds, sentences)
# print(seeds)

patterns = {"m_sg" : "A ingestão abundante de tomate e seus derivados diminui em 35% os riscos de câncer da próstata, segundo [MASK] realizado na Universidade de Harvard.", 
            "m_pl" : "A ingestão abundante de tomate e seus derivados diminui em 35% os riscos de câncer da próstata, segundo [MASK] realizados na Universidade de Harvard.", 
            "f_sg" : "A ingestão abundante de tomate e seus derivados diminui em 35% os riscos de câncer da próstata, segundo [MASK] realizada na Universidade de Harvard.", 
            "f_pl" : "A ingestão abundante de tomate e seus derivados diminui em 35% os riscos de câncer da próstata, segundo [MASK] realizadas na Universidade de Harvard."}
seeds = {}
for pattern in patterns:
  seeds[pattern] = []

for pattern in patterns:
    sent = patterns[pattern]
    words = nlp_fill(sent)
    # art = sent.split("[MASK]")[0].strip().split()[-1]
    # print(sent)
    for word in words:
      if word["score"] < 0.001:
        continue
      # print("\t",word["token_str"], word["score"])
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      word = word["token_str"].lower()
      if word not in seeds[pattern]:
        seeds[pattern].append(word)
sentences = ["A ingestão abundante de tomate e seus derivados diminui em 35% os riscos de câncer da próstata, segundo <SEED> [MASK] na Universidade de Harvard."]

TASKS["concordância nominal_1c"] = (seeds, sentences)

patterns = {"sg" : "Entre outras medidas tomadas pelo governo Itamar, se destacou a decisão de fomentar a produção de [MASK] populares.", 
            "pl" : "Entre outras medidas tomadas pelo governo Itamar, se destacou a decisão de fomentar a produção de [MASK] popular."}
seeds = {}
for pattern in patterns:
  seeds[pattern] = []

for pattern in patterns:
    sent = patterns[pattern]
    words = nlp_fill(sent)
    # art = sent.split("[MASK]")[0].strip().split()[-1]
    # print(sent)
    for word in words:
      if word["score"] < 0.001:
        continue
      # print("\t",word["token_str"], word["score"])
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      word = word["token_str"].lower()
      if word not in seeds[pattern]:
        seeds[pattern].append(word)
sentences = ["Entre outras medidas tomadas pelo governo Itamar, se destacou a decisão de fomentar a produção de <SEED> [MASK]."]

TASKS["concordância nominal_1d"] = (seeds, sentences)


patterns = {"m_sg" : "Outro problema é a desertificação, ocasionada pelo uso inadequado do solo, bem como pelas queimadas sucessivas e pelo [MASK] intenso", 
            "m_pl" : "Outro problema é a desertificação, ocasionada pelo uso inadequado do solo, bem como pelas queimadas sucessivas e pelos [MASK] intensos.", 
            "f_sg" : "Outro problema é a desertificação, ocasionada pelo uso inadequado do solo, bem como pelas queimadas sucessivas e pela [MASK] intensa.", 
            "f_pl" : "Outro problema é a desertificação, ocasionada pelo uso inadequado do solo, bem como pelas queimadas sucessivas e pelas [MASK] intensas."}
seeds = {}
for pattern in patterns:
  seeds[pattern] = []

for pattern in patterns:
    sent = patterns[pattern]
    words = nlp_fill(sent)
    art = sent.split("[MASK]")[0].strip().split()[-1:]
    art = " ".join(art)
    # print(sent)
    for word in words:
      if word["score"] < 0.001:
        continue
      # print("\t",word["token_str"], word["score"])
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      word = art + " " + word["token_str"].lower()
      if word not in seeds[pattern]:
        seeds[pattern].append(word)
sentences = ["Outro problema é a desertificação, ocasionada pelo uso inadequado do solo, bem como pelas queimadas sucessivas e <SEED> [MASK]."]

TASKS["concordância nominal_1e"] = (seeds, sentences)




In [ ]:
patterns = {"m_sg" : "É precisamente na revelação do [MASK] humano que reside o valor dessas cartas, diz.", 
            "m_pl" : "É precisamente na revelação dos [MASK] humanos que reside o valor dessas cartas, diz.", 
            "f_sg" : "É precisamente na revelação da [MASK] humana que reside o valor dessas cartas, diz.", 
            "f_pl" : "É precisamente na revelação das [MASK] humanas que reside o valor dessas cartas, diz."}
seeds = {}
for pattern in patterns:
  seeds[pattern] = []

for pattern in patterns:
    sent = patterns[pattern]
    words = nlp_fill(sent)
    art1 = sent.split("[MASK]")[0].strip().split()[-1]
    # art2 =  sent.split("[MASK]")[1].strip().split()[0]
    # art = " ".join(art)
    # print(sent)
    for word in words:
      if word["score"] < 0.001:
        continue
      # print("\t",word["token_str"], word["score"])
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      word = art1 + " " + word["token_str"].lower() # + " " + art2
      if word not in seeds[pattern]:
        seeds[pattern].append(word)
sentences = ["É precisamente na revelação <SEED> [MASK] que reside o valor dessas cartas, diz."]
seeds["f_sg"].append("da subjetividade")
TASKS["concordância nominal_2a"] = (seeds, sentences)


patterns = {"m_sg" : "Foi um [MASK] tenso, seguida de uma entrevista da qual também participaram investidores.", 
            "m_pl" : "Foram uns [MASK] tensos, seguida de uma entrevista da qual também participaram investidores.", 
            "f_sg" : "Foi uma [MASK] tensa, seguida de uma entrevista da qual também participaram investidores.", 
            "f_pl" : "Foram umas [MASK] tensas, seguida de uma entrevista da qual também participaram investidores."}
seeds = {}
for pattern in patterns:
  seeds[pattern] = []

for pattern in patterns:
    sent = patterns[pattern]
    words = nlp_fill(sent)
    art1 = sent.split("[MASK]")[0].strip()
    # art2 =  sent.split("[MASK]")[1].strip().split()[0]
    # print(sent)
    for word in words:
      if word["score"] < 0.001:
        continue
      # print("\t",word["token_str"], word["score"])
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      word = art1 + " " + word["token_str"].lower() #+ " " + art2
      if word not in seeds[pattern]:
        seeds[pattern].append(word)
sentences = ["<SEED> [MASK] seguida de uma entrevista da qual também participaram investidores."]
seeds["f_sg"].append("Foi uma palestra")
TASKS["concordância nominal_2b"] = (seeds, sentences)

patterns = {"m_sg" : "E sua ação patriótica, ao lado da gloriosa Marinha de Guerra, no [MASK] duro que travamos contra os exércitos dos ditadores do Prata, ao tempo em que nossa gente enfrentou as aguerridas legiões paraguaias, que haviam invadido ousadamente Mato Grosso e o Rio Grande do Sul.", 
            "m_pl" : "E sua ação patriótica, ao lado da gloriosa Marinha de Guerra, nos [MASK] duros que travamos contra os exércitos dos ditadores do Prata, ao tempo em que nossa gente enfrentou as aguerridas legiões paraguaias, que haviam invadido ousadamente Mato Grosso e o Rio Grande do Sul.", 
            "f_sg" : "E sua ação patriótica, ao lado da gloriosa Marinha de Guerra, na [MASK] dura que travamos contra os exércitos dos ditadores do Prata, ao tempo em que nossa gente enfrentou as aguerridas legiões paraguaias, que haviam invadido ousadamente Mato Grosso e o Rio Grande do Sul.", 
            "f_pl" : "E sua ação patriótica, ao lado da gloriosa Marinha de Guerra, nas [MASK] duras que travamos contra os exércitos dos ditadores do Prata, ao tempo em que nossa gente enfrentou as aguerridas legiões paraguaias, que haviam invadido ousadamente Mato Grosso e o Rio Grande do Sul."}
seeds = {}
for pattern in patterns:
  seeds[pattern] = []

for pattern in patterns:
    sent = patterns[pattern]
    words = nlp_fill(sent)
    art = sent.split("[MASK]")[0].strip().split()[-1]
    # print(sent)
    for word in words:
      if word["score"] < 0.001:
        continue
      # print("\t",word["token_str"], word["score"])
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      word = art + " " + word["token_str"].lower()
      if word not in seeds[pattern]:
        seeds[pattern].append(word)
sentences = ["E sua ação patriótica, ao lado da gloriosa Marinha de Guerra, <SEED> [MASK] que travamos contra os exércitos dos ditadores do Prata, ao tempo em que nossa gente enfrentou as aguerridas legiões paraguaias, que haviam invadido ousadamente Mato Grosso e o Rio Grande do Sul."]
seeds["f_pl"].append("nas batalhas")
TASKS["concordância nominal_2c"] = (seeds, sentences)

patterns = {"m_sg" : "A Astra e a Zeneca terão lucro [MASK] totais de cerca de US$ 15,9 bilhões.", 
            "m_pl" : "A Astra e a Zeneca terão lucros [MASK] totais de cerca de US$ 15,9 bilhões.", 
            "f_sg" : "A Astra e a Zeneca terão venda [MASK] totais de cerca de US$ 15,9 bilhões.", 
            "f_pl" : "A Astra e a Zeneca terão vendas [MASK] totais de cerca de US$ 15,9 bilhões."}
seeds = {}
for pattern in patterns:
  seeds[pattern] = []

for pattern in patterns:
    sent = patterns[pattern]
    words = nlp_fill(sent)
    art = sent.split("[MASK]")[0].strip().split()[-1]
    # print(sent)
    for word in words:
      if word["score"] < 0.001:
        continue
      # print("\t",word["token_str"], word["score"])
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      word = art + " " + word["token_str"].lower()
      if word not in seeds[pattern]:
        seeds[pattern].append(word)
sentences = ["A Astra e a Zeneca terão <SEED> [MASK] de cerca de US$ 15,9 bilhões."]
seeds["f_pl"].append("vendas")
TASKS["concordância nominal_2c"] = (seeds, sentences)

patterns = {"m_sg" : "Nada mais incômodo do que não rir do [MASK] velhas de um palhaço profissional.", 
            "m_pl" : "Nada mais incômodo do que não rir dos [MASK] velhas de um palhaço profissional.", 
            "f_sg" : "Nada mais incômodo do que não rir da [MASK] velhas de um palhaço profissional.", 
            "f_pl" : "Nada mais incômodo do que não rir das [MASK] velhas de um palhaço profissional."}
seeds = {}
for pattern in patterns:
  seeds[pattern] = []

for pattern in patterns:
    sent = patterns[pattern]
    words = nlp_fill(sent)
    art = sent.split("[MASK]")[0].strip().split()[-1]
    # print(sent)
    for word in words:
      if word["score"] < 0.001:
        continue
      # print("\t",word["token_str"], word["score"])
      if len(word["token_str"])>2 and "##" == word["token_str"][:2]:
        continue
      word = art + " " + word["token_str"].lower()
      if word not in seeds[pattern]:
        seeds[pattern].append(word)
sentences = ["Nada mais incômodo do que não rir <SEED> [MASK] de um palhaço profissional."]
seeds["f_pl"].append("das piadas")
TASKS["concordância nominal_2c"] = (seeds, sentences)


##templates 5 - voz passiva.txt

In [ ]:
seeds = {"dever":["deve ser","deveria","deverá","poderia", "poderá","pôde", "podia"],
          "ter":["teria de","terá de", "vai ter de", "teve de", "tinha de"],
          "precisar":["precisaria","precisará","vai precisar", "precisou","precisava"],
          "necessitar":["necessitaria","necessitará","vai necessitar","necessitou","necessitava"]}
sentences = ["A ocorrência de náusea, dor abdominal, diarréia, flatulência e reações cutâneas <SEED> [MASK] ao médico, assim como o aparecimento de qualquer outra reação desagradável."]
TASKS["voz passiva_1a"] = (seeds, sentences)

seeds = {"none":[""]}
sentences = ["Porque Brasília nasceu sendo [MASK]."]
TASKS["voz passiva_1b"] = (seeds, sentences)


seeds = {"single_pattern":["possa", "tenha de", "deva", "precise", "venha a"]}
sentences = ["Após a limpeza da mata, uma área de roça é utilizada durante 4 ou 5 anos e depois deixada de lado por um período de 7 a 10 anos antes que <SEED> ser [MASK]."]
TASKS["voz passiva_1c"] = (seeds, sentences)

seeds = {"single_pattern":["sejam","fossem","possam ser","pudessem ser","acabem sendo","acabassem sendo"]}
sentences = ["Não se tolera mais, por exemplo, que uma briga pessoal culmine com um duelo do qual um dos envolvidos saia morto, como acontecia na Idade Média, nem que pessoas <SEED> [MASK] em praça pública, como acontecia há alguns séculos (LOCONTE, 1999)."]
TASKS["voz passiva_1d"] = (seeds, sentences)

seeds = {"single_pattern":["podendo ser","sendo","vindo a ser"]}
sentences = ["Também está presente em todos os espectros um sinal de absorção centrado em 1.025 cm-1, <SEED> [MASK] à presença de carboidratos nos AF e AH."]
TASKS["voz passiva_1e"] = (seeds, sentences)


In [ ]:
seeds = {"ser":["ia ser","seria","iria ser","poderia ser","viria a ser","acabaria sendo"]}
sentences = ["Porque ele dizia para mim que o meu pai <SEED> [MASK] ou ia morrer se eu contasse alguma coisa."]
TASKS["voz passiva_2a"] = (seeds, sentences)

seeds = {"ser":["eram","acabavam sendo","podiam ser","deviam ser","deveriam ser","precisavam ser","costumavam ser"]}
sentences = ["Contrariando ao ideal materno dos romanos, os filhos das classes médias e altas <SEED> [MASK] às amas de leite que os alimentavam e se incumbiam dos cuidados gerais da criança."]
TASKS["voz passiva_2b"] = (seeds, sentences) 

seeds = {"ser":["passa a ser","passou a ser", "passará a ser", "vai passar a ser", "passaria a ser", "é", "foi", "será", "vai ser", "seria", "pode ser", "pôde ser", "poderia ser", "poderá ser", "vem a ser", "viria a ser", "veio a ser", "virá a ser", "vai vir a ser", "acaba sendo", "acabará sendo", "acabou sendo", "vai acabar sendo", "acabaria sendo"]}
sentences = ["De acordo com essa nova postura acolhida por muitos dos pesquisadores da ASL, o aprendiz de línguas <SEED> [MASK] como um ser complexo - detentor de uma identidade mutável através de tempo e espaço."]
TASKS["voz passiva_2c"] = (seeds, sentences) 

seeds = {"ser":["devem ser","foram", "seriam", "são", "serão", "vão ser", "deveriam ser", "deviam ser", "deverão ser", "podem", "poderiam ser", "puderam ser", "poderão ser", "vão poder ser", "tiveram de ser", "têm de ser", "teriam de ser", "terão de ser", "vão ter de ser", "vieram a ser", "viriam a ser", "virão a ser", "vão vir a ser", "acabaram sendo", "acabarão sendo", "acabariam sendo", "terão sido", "vão ter sido"]}
sentences = ["Antiinflamatórios, aspirina e anti-coagulantes <SEED> [MASK] 7 a 10 dias antes da cirurgia."]
TASKS["voz passiva_2d"] = (seeds, sentences)  

seeds = {"ser":["deveria ter sido","será","seria", "vai ser", "foi", "é", "tem sido", "teria sido", "poderia ter sido", "pode ter sido", "pode ser", "poderá ser", "vai poder ser", "veio a ser", "virá a ser", "acabou sendo", "acaba sendo", "acabará sendo", "vai acabar sendo"]}
sentences = ["Ele <SEED> [MASK] a coronhadas."]
TASKS["voz passiva_2e"] = (seeds, sentences)  

seeds = {"ser":["vai ser","será", "é", "virá a ser", "vai vir a ser", "acabará sendo", "acaba sendo", "vai acabar sendo", "pode ser", "poderá ser", "vai poder ser"]}
sentences = ["A redação da lei não mais se refere a empresas com mais de 200 empregados, nem o obreiro <SEED> [MASK] para promover o entendimento direto com os empregadores."]
TASKS["voz passiva_2f"] = (seeds, sentences)  

seeds = {"ser":["era", "seria", "deveria ser", "devia ser", "teria de ser", "tinha de ser", "precisava ser", "precisaria ser", "necessitava ser", "necessitaria ser"]}
sentences = ["Esse título vem de 'One if by land, two if by sea', frase usada pelos franceses para quando um pelotão inglês se aproxima pela terra, <SEED> [MASK] um lampião, e quando eles se aproximavam pelo mar, dois."]
TASKS["voz passiva_2g"] = (seeds, sentences)  

seeds = {"ser":["eram", "seriam", "foram", "teriam sido", "serão", "terão sido", "vão ser", "devem ser", "deveriam ser", "deverão ser", "podem ser", "poderão ser", "vão poder ser", "poderiam ser", "puderam ser", "podiam ser", "precisam ser", "precisariam ser", "precisarão ser", "vão precisar ser", "precisaram ser", "precisavam ser", "necessitam ser", "necessitariam ser", "necessitaram ser", "necessitarão ser", "nacessitavam ser", "tinham de ser", "têm de ser", "tiveram de ser", "terão de ser", "vão ter de ser", "teriam de ser"]}
sentences = ["No dia 21 de março, às 20 horas, <SEED> [MASK] os refletores do campo."]
TASKS["voz passiva_2h"] = (seeds, sentences)  


##templates 6 - conectores.txt

In [ ]:
seeds = {"none":[""]}
sentences = ["Quanto ao planejamento de negócios, o Balanced Scorecard tem grande impacto [MASK] é desdobrado para promover as mudanças organizacionais.",
            "Cerca de 1,45 milhão de contas, totalizando R$ 390 milhões, foram desativadas em novembro de 97, [MASK] se encerrou o prazo dado pelo BC para o recadastramento.",
            "O consultor de meio-ambiente boliviano William Savedra, 36, diz ter ouvido a palavra \"coitado\" pela primeira vez [MASK], recém-chegado ao Brasil, foi jantar sozinho em um restaurante."]
TASKS["conectores_quando"] = (seeds, sentences)



seeds = {"none":[""]}
sentences = ["Telê decidiu poupar alguns jogadores [MASK] quer o time com força máxima para partida contra o Grêmio, sexta-feira, em Porto Alegre, pela Copa do Brasil (leia texto nesta página).",
            "Mas nós não tínhamos este intercâmbio cultural integralista entre as cidades da vizinhança, [MASK] nossos núcleos eram jovens ainda." ,
            "Alguém vai ter que pagar por isso, [MASK] vai contra o esforço de estabilização." ]
TASKS["conectores_porque"] = (seeds, sentences)


seeds = {"none":[""]}
sentences = ["A diretoria do clube paraense ameaça afastar alguns jogadores [MASK] o time não vencer o Cruzeiro.",
            "[MASK] o coeficiente de aproveitamento único para a cidade é de uma vez a área do terreno, por lei, poderiam ser construídos quatro pavimentos.",
            "[MASK] a matriz W, além de positiva, for simétrica (como no caso da distância de Mahalanobis) a matriz W pode ser escrita como:"]
TASKS["conectores_Se"] = (seeds, sentences)


seeds = {"none":[""]}
sentences = ["O etomidato, outro anestésico, foi considerado seguro para ser administrado em pacientes graves, [MASK] os reflexos do sistema nervoso autônomo são preservados, assim como a contratilidade do miocárdio.",
            "Não seria o caso de continuar pensado em subdivisões, [MASK] isso geraria uma quantidade de subgrupos com falou se tivesse mais escola ao mencionar a frase Quanto antes você começar mais chance você terá, está se referindo às oportunidades que o candidato terá de aperfeiçoamento profissional no exterior se possuir um \"currículo altamente competitivo\".",
            "Apresentaram, inclusive, certos hábitos predadores, [MASK] algumas das ninfas foram observadas sugando formas análogas de blatídeos Blaberinae, de espécie comumente encontrada no local."]
TASKS["conectores_pois"] = (seeds, sentences)


seeds = {"none":[""]}
sentences = ["[MASK], esse novo modelo de Estado continuou atrelado aos dizeres capitalistas, lembre-se que o próprio Keynes definia-se como um capitalista avançado.",
            "A importância desses testes no diagnóstico precoce da doença aterosclerótica, [MASK], permanece indefinida.",
            "[MASK] o referido processo já foi objeto de estudo nos pareceres supracitados, cabendo apenas à CAPLAN fixar o número de vagas."]
TASKS["conectores_Contudo"] = (seeds, sentences)


seeds = {"none":[""]}
sentences = ["Yoko adultos para efeito de recebimento de dinheiro do FUNDE, e outros pontos, como a não utilização das verbas do salário-educação para complementação do fundo por parte da União, garantindo que esse dinheiro fosse utilizado como verba adicional, [MASK] prevê a Constituição, e também a repartição de 70 % da quota estadual do salário-educação entre os Estados e seus Municípios, conforme o número de matrículas no ensino fundamental, o PT votou favoravelmente ao projeto.",
              "Entretanto, a comunidade usuária e produtora de conhecimento sobre o Jogo de Areia tem demonstrado, ao longo dos últimos anos, preocupação com relação ao seu estágio de desenvolvimento no âmbito cientifico da psicologia clínica, [MASK] afirmou Ammann (1997):",
              "Contudo, [MASK] se verificou, apesar de a equiparação ter sido concedida a partir de janeiro de 1961, os primeiros comissionamentos femininos só ocorreram a partir de meados de 1964."]
TASKS["conectores_conforme"] = (seeds, sentences)


seeds = {"none":[""]}
sentences = ["[MASK] isso, dos oito Colaboradores Leigos que responderam, seis disseram ser ela necessária, um que era viável, e um não o saber.",
            "O índice de consumo de energia mensal, para a estação de bombeamento booster foi obtido por meio da razão entre o consumo de energia registrado na referida estação de bombeamento e o somatório do volume mensal de água registrados nos hidrômetros, [MASK] o índice de consumo de energia mensal para todo perímetro o foi através da razão entre o somatório dos consumos mensais registrados nas estações de bombeamento e o somatório do volume de água mensal registrado nos hidrômetros.",
            "[MASK] ele falava, os moradores começaram novamente a se agrupar à sua volta e dizer que ele tinha que ir embora para não apanhar mais."]
TASKS["conectores_Enquanto"] = (seeds, sentences)


seeds = {"none":[""]}
sentences = ["A cada dia surge uma informação diferente, [MASK] beneficiando os servidores, ora prejudicando-os;",
              "Dir-se-á que incidem sob a mesma crítica os substitutivos anteriores, provindos ora do Executivo, [MASK] do Legislativo.",
              "No citoplasma, as moléculas de ácido araquidônico podem sofrer ora a ação da cicloxigenase, outra enzima do citosol ativada pelo maior afluxo de cálcio, [MASK] a ação da lipoxigenase, gerando respectivamente as prostaglandinas e os leucotrienos."]
TASKS["conectores_ora"] = (seeds, sentences)


seeds = {"none":[""]}
sentences = ["Não houve diferença entre os índices de respostas adequadas ou não, quando analisado por especialidade - clínicos e cirurgiões índices de acordo com o tempo de formado, observa-se que os médicos com mais de dez anos de graduação, apresentam maior tendência a reduzir os níveis tensionais, quando comparados com aqueles com até cinco anos de formado DISCUSSÃO O cérebro, ao contrário de outros tecidos, armazena pouca glicose, [MASK], necessita de fluxo sanguíneo contínuo.",
            'Verifica-se, [MASK], que a inclusão das variáveis "nem sempre a recíproca / o contrário é verdadeirao)" juntamente com a alternativa "pode ser ou pode não ser" tiveram um efeito positivo no número de acertos dos sujeitos universitários nas formas inválidas, embora tenham tido um efeito negativo nas formas válidas.',
            "[MASK], não se deve limitar esse ouvir apenas a escutar."]
TASKS["conectores_Portanto"] = (seeds, sentences)


seeds = {"none":[""]}
sentences = ["Não escreve o roteiro [MASK] dirige os atores.",
              "Isso porque a erva-mate pode permanecer nas lavouras remanescentes de velhas derrubadas sem prejudicar o desenvolvimento de qualquer outra planta, [MASK] é prejudicada pelos animais, podendo, por isso, ser cultivada mesmo nos campos destinados ao pastoreio;",
              "O servidor que ganha mais do que esse patamar não terá direito a se aposentar com o salário integral [MASK] receberá 100 % dos reajustes e vantagens repassadas pelo governo aos servidores da ativa."]
TASKS["conectores_nem"] = (seeds, sentences)


seeds = {"none":[""]}
sentences = ["[MASK], não basta passar por essa barreira para concretizar aspirações de sucesso profissional.",
            "[MASK], no aspecto teórico, não se pode desprezar a possibilidade da pintura enquanto uma linguagem para representação do mundo e de intervenção nele, afinal a produção cultural não é, em princípio, utilitária.",
            "[MASK], o avanço das nossas discussões permitiu um acordo, qual seja , o estabelecimento de um tipo de pena que avança sobre a prerrogativa do mandato, a ser aplicada pelo Plenário da Câmara dos Deputados."]
TASKS["conectores_Todavia"] = (seeds, sentences) 


seeds = {"none":[""]}
sentences = ["Como o interferômetro GT tem um comportamento periódico, [MASK] a peridiocidade seja ajustada de acordo com o espaçamento entre canais, um único dispositivo pode ser usado para compensar os canais de um sistema WDM.",
            "Se a ressurreição de Jesus há de ser a vitória escatológica da fé efetiva (ainda que livre) nela, uma fé na qual culmina a própria natureza da ressurreição.85 Esta posição de Rahner tem uma conseqüência importante para a hermenêutica pois se não aceita a possibilidade de que Deus possa, por princípio, realizar um acontecimento escatológico na história e, [MASK] o realize, o mesmo não possa ser reconhecido como tal, assim não podemos compreender a ressurreição.",
            "[MASK] o nome tenha mais que 70 caracteres, abreviar o suficiente para que caiba nas 70 posições do campo no formulário."]
TASKS["conectores_Caso"] = (seeds, sentences) 



#Manual process

In [ ]:
TASKS

{'concordancia_de_sujeito_1_adotar': ({'Condicional Futuro do Pretérito Simples_2pl': ['adotaríeis'],
   'Condicional Futuro do Pretérito Simples_pl': ['adotaríamos', 'adotariam'],
   'Condicional Futuro do Pretérito Simples_sg': ['adotaria', 'adotarias'],
   'Indicativo Futuro do Presente Simples_2pl': ['adotareis'],
   'Indicativo Futuro do Presente Simples_pl': ['adotaremos', 'adotarão'],
   'Indicativo Futuro do Presente Simples_sg': ['adotarei',
    'adotarás',
    'adotará'],
   'Indicativo Pretérito Mais que Perfeito Simples_2pl': ['adotáreis'],
   'Indicativo Pretérito Mais que Perfeito Simples_pl': ['adotaram',
    'adotáramos'],
   'Indicativo Pretérito Mais que Perfeito Simples_sg': ['adotara',
    'adotaras'],
   'Indicativo presente_2pl': ['adotais'],
   'Indicativo presente_pl': ['adotamos', 'adotam'],
   'Indicativo presente_sg': ['adota', 'adoto', 'adotas'],
   'Indicativo pretérito imperfeito_2pl': ['adotáveis'],
   'Indicativo pretérito imperfeito_pl': ['adotávamos', 

In [ ]:
# json.dump(TASKS, open("tasks.automatically_generated.json", "w"))

## offline review step

In [ ]:
TASKS = eval(open("/content/tasks.automatically_generated_May25-Corrigido_official.txt", 'r').read())

In [ ]:
TASKS

{'concordancia_de_sujeito_1_adotar': ({'Condicional Futuro do Pretérito Simples_2pl': ['adotaríeis'],
   'Condicional Futuro do Pretérito Simples_pl': ['adotaríamos', 'adotariam'],
   'Condicional Futuro do Pretérito Simples_sg': ['adotarias', 'adotaria'],
   'Indicativo Futuro do Presente Simples_2pl': ['adotareis'],
   'Indicativo Futuro do Presente Simples_pl': ['adotaremos', 'adotarão'],
   'Indicativo Futuro do Presente Simples_sg': ['adotarei',
    'adotará',
    'adotarás'],
   'Indicativo Pretérito Mais que Perfeito Simples_2pl': ['adotáreis'],
   'Indicativo Pretérito Mais que Perfeito Simples_pl': ['adotáramos',
    'adotaram'],
   'Indicativo Pretérito Mais que Perfeito Simples_sg': ['adotara',
    'adotaras'],
   'Indicativo presente_2pl': ['adotais'],
   'Indicativo presente_pl': ['adotamos', 'adotam'],
   'Indicativo presente_sg': ['adoto', 'adota', 'adotas'],
   'Indicativo pretérito imperfeito_2pl': ['adotáveis'],
   'Indicativo pretérito imperfeito_pl': ['adotávamos', 

In [ ]:
print(len(TASKS))

83


#test

In [ ]:
model_name = "neuralmind/bert-base-portuguese-cased"
model_short_name = "base"
nlp_fill = pipeline('fill-mask',model=model_name, use_fast=True, top_k=10)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
%%time
use_cls = False
use_sep = False

results_pos_compressed = {}
results_ood_compressed = {}
outputs_by_sent_seed = {}
for task in TASKS:
  print("...",task)
  results_scores, results_pos, results_ood, outputs = process(seeds=TASKS[task][0], sentences=TASKS[task][1], use_cls=use_cls, use_sep=use_sep, nlp_fill=nlp_fill, pos_dict=pos_dict, task=task, verbose = False)
  for key in outputs:
    outputs_by_sent_seed[key] = outputs[key]
  for value in results_ood:
    results_ood_compressed[task + "_" + value] = results_ood[value]
  
  task_cnt = Counter()
  for value in results_pos:
    for s in results_pos[value]:
      cnt = results_pos[value][s]
      for k in cnt:
        # print(s, results_pos[s])
        task_cnt[k] += cnt[k]
    results_pos_compressed[task + "_" + value] = task_cnt

... concordancia_de_sujeito_1_adotar
... concordancia_de_sujeito_1_afirmar
... concordancia_de_sujeito_1_comentar
... concordancia_de_sujeito_1_estar
... concordancia_de_sujeito_1_justificar
... concordancia_de_sujeito_1_ser
... concordancia_de_sujeito_2_carregar
... concordancia_de_sujeito_2_contribuir
... concordancia_de_sujeito_2_falar
... concordancia_de_sujeito_2_fazer
... concordância nominal_1a
... concordância nominal_1b
... concordância nominal_1c
... concordância nominal_1d
... concordância nominal_1e
... concordância nominal_2a
... concordância nominal_2b
... concordância nominal_2c
... concordância verbal_1_0
... concordância verbal_1_1
... concordância verbal_1_2
... concordância verbal_1_3
... concordância verbal_2_0
... concordância verbal_2_1
... concordância verbal_2_2
... concordância verbal_2_3
... concordância verbal_3a
... concordância verbal_3b
... concordância verbal_3c
... concordância verbal_3d
... concordância verbal_4_0
... concordância verbal_4_1
... concord

In [ ]:
with open("/content/drive/MyDrive/Aline et al/results_cls" + str(use_cls) + "_" + model_short_name + ".csv", "w") as output_file:
  sep = "\t"
  output_file.write("template"+sep+"seed\tsentence" +sep+ "all_pos" +sep+ "token_str" +sep+ "token_score" +sep+"wrong_pos"+sep+"lexis_comment" + "\n")
  for key in outputs_by_sent_seed:
    for all_pos, token_str, score in outputs_by_sent_seed[key]:
      # print(key,all_pos, token_str, score)
      output_file.write(key +sep+ all_pos +sep+ token_str +sep+ str(score) + "\n")


In [ ]:
for task in results_pos_compressed:
  print(task)
  print("\t", results_pos_compressed[task])
  print("\tOOD",results_ood_compressed[task])

concordancia_de_sujeito_1_adotar_Indicativo pretérito perfeito simples_sg
	 Counter({'ADV': 10, 'N:ms': 8, 'CONJ': 7, 'V:P3s': 5, 'N:fs': 5, 'V:Y2s': 5, 'PRO+Pes:A3fs': 4, 'PRO+Pes:N3fs': 3, 'PRO+Pes:R3ms': 2, 'PRO+Pes:R3fp': 2, 'PRO+Pes:R3mp': 2, 'PRO+Pes:R3fs': 2, 'V:S3s': 2, 'V:S1s': 2, 'PRO+Pes:A3ms': 2, 'V:Y3s': 2, 'PRO+Pes:N3ms': 2, 'PRO+Dem:fs': 2, 'PRO+Pes:N2fs': 2, 'PRO+Pes:N2ms': 2, 'PRO+Pes:N1fs': 2, 'PRO+Pes:N1ms': 2, 'PREPXPRO+Pes:O3fs': 2, 'N+Pr:fs': 1, 'PRO+Tra:3fs': 1, 'PRO+Ind:ms': 1, 'PRO+Ind:fs': 1, 'PRO+Tra:3ms': 1, 'A:fs': 1, 'PRO+Dem:ms': 1, 'A:ms': 1, 'PREP': 1, 'DET+Art+Def:fs': 1, 'PRO+Pes:D3mp': 1, 'PRO+Pes:O1mp': 1, 'PRO+Pes:R1fp': 1, 'PREPXPRO+Dem:mp': 1, 'PRO+Pes:D1mp': 1, 'PRO+Pes:O1fp': 1, 'PRO+Pes:O3mp': 1, 'PRO+Pes:R1mp': 1, 'PREPXDET+Art+Def:mp': 1, 'PRO+Pes:A1mp': 1, 'PRO+Pes:A3mp': 1, 'PRO+Pes:D1fp': 1, 'PRO+Pes:A1fp': 1, 'PRO+Pes:O3fs': 1, 'PRO+Pes:O3ms': 1, 'PRO+Pes:D3fs': 1, 'PRO+Pes:D3ms': 1})
	OOD set()
concordancia_de_sujeito_1_afirmar_Indicati

In [ ]:
model_name = "neuralmind/bert-large-portuguese-cased"
model_short_name = "large"
nlp_fill = pipeline('fill-mask',model=model_name, use_fast=True, top_k=10)

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
%%time
use_cls = False
use_sep = False


results_pos_compressed = {}
results_ood_compressed = {}
outputs_by_sent_seed = {}
for task in TASKS:
  print("...",task)
  results_scores, results_pos, results_ood, outputs = process(seeds=TASKS[task][0], sentences=TASKS[task][1], use_cls=use_cls, use_sep=use_sep, nlp_fill=nlp_fill, pos_dict=pos_dict, task=task, verbose = False)
  for key in outputs:
    outputs_by_sent_seed[key] = outputs[key]
  for value in results_ood:
    results_ood_compressed[task + "_" + value] = results_ood[value]
  
  task_cnt = Counter()
  for value in results_pos:
    for s in results_pos[value]:
      cnt = results_pos[value][s]
      for k in cnt:
        # print(s, results_pos[s])
        task_cnt[k] += cnt[k]
    results_pos_compressed[task + "_" + value] = task_cnt

... concordancia_de_sujeito_1_adotar
... concordancia_de_sujeito_1_afirmar
... concordancia_de_sujeito_1_comentar
... concordancia_de_sujeito_1_estar
... concordancia_de_sujeito_1_justificar
... concordancia_de_sujeito_1_ser
... concordancia_de_sujeito_2_carregar
... concordancia_de_sujeito_2_contribuir
... concordancia_de_sujeito_2_falar
... concordancia_de_sujeito_2_fazer
... concordância nominal_1a
... concordância nominal_1b
... concordância nominal_1c
... concordância nominal_1d
... concordância nominal_1e
... concordância nominal_2a
... concordância nominal_2b
... concordância nominal_2c
... concordância verbal_1_0
... concordância verbal_1_1
... concordância verbal_1_2
... concordância verbal_1_3
... concordância verbal_2_0
... concordância verbal_2_1
... concordância verbal_2_2
... concordância verbal_2_3
... concordância verbal_3a
... concordância verbal_3b
... concordância verbal_3c
... concordância verbal_3d
... concordância verbal_4_0
... concordância verbal_4_1
... concord

In [ ]:
with open("/content/drive/MyDrive/Aline et al/results_cls" + str(use_cls) + "_" + model_short_name + ".csv", "w") as output_file:
  sep = "\t"
  output_file.write("template"+sep+"seed\tsentence" +sep+ "all_pos" +sep+ "token_str" +sep+ "token_score" +sep+"wrong_pos"+sep+"lexis_comment" + "\n")
  for key in outputs_by_sent_seed:
    for all_pos, token_str, score in outputs_by_sent_seed[key]:
      # print(key,all_pos, token_str, score)
      output_file.write(key +sep+ all_pos +sep+ token_str +sep+ str(score) + "\n")


In [ ]:
for task in results_pos_compressed:
  print(task)
  print("\t", results_pos_compressed[task])
  print("\tOOD",results_ood_compressed[task])

concordancia_de_sujeito_1_adotar_Indicativo pretérito perfeito simples_sg
	 Counter({'ADV': 8, 'PRO+Pes:A3fs': 6, 'N:fs': 5, 'PRO+Dem:fs': 5, 'N:ms': 5, 'PREPXPRO+Pes:O3fs': 4, 'V:P3s': 3, 'PRO+Pes:N3fs': 3, 'V:Y2s': 3, 'CONJ': 3, 'PREP': 3, 'DET+Art+Def:fs': 3, 'PRO+Pes:N2fs': 2, 'PRO+Pes:N2ms': 2, 'PRO+Pes:R3ms': 1, 'PRO+Pes:R3fp': 1, 'PRO+Pes:R3mp': 1, 'PRO+Pes:R3fs': 1, 'V:S3s': 1, 'V:S1s': 1, 'PRO+Pes:A3ms': 1, 'V:Y3s': 1, 'PRO+Pes:N3ms': 1, 'A:fs': 1, 'PRO+Dem:ms': 1, 'A:ms': 1, 'N+Pr:fs': 1, 'N+Pr:ms': 1, 'PRO+Pes:N1fp': 1, 'PRO+Pes:N1mp': 1, 'N:mp': 1, 'PRO+Pes:N1fs': 1, 'PRO+Pes:N1ms': 1})
	OOD set()
concordancia_de_sujeito_1_afirmar_Indicativo pretérito perfeito simples_sg
	 Counter({'ADV': 12, 'N:ms': 11, 'CONJ': 10, 'N+Pr:ms': 6, 'N+Pr:fs': 4, 'PRO+Pes:N2fs': 3, 'PRO+Pes:N2ms': 3, 'PRO+Ind:ms': 3, 'N:fs': 3, 'V:P1s': 2, 'PRO+Rel:ms': 2, 'PRO+Pes:N1fs': 2, 'PRO+Pes:N1ms': 2, 'V:P3s': 2, 'V:Y2s': 2, 'N+Pr:fp': 2, 'N+Pr:mp': 2, 'PRO+Tra:3fs': 1, 'PRO+Ind:fs': 1, 'PRO+Tra:3ms':

In [ ]:
model_name = "bert-base-multilingual-cased"
model_short_name = "mbert"
nlp_fill = pipeline('fill-mask',model=model_name, use_fast=True, top_k=10)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
%%time
use_cls = False
use_sep = False


results_pos_compressed = {}
results_ood_compressed = {}
outputs_by_sent_seed = {}
for task in TASKS:
  print("...",task)
  results_scores, results_pos, results_ood, outputs = process(seeds=TASKS[task][0], sentences=TASKS[task][1], use_cls=use_cls, use_sep=use_sep, nlp_fill=nlp_fill, pos_dict=pos_dict, task=task, verbose = False)
  for key in outputs:
    outputs_by_sent_seed[key] = outputs[key]
  for value in results_ood:
    results_ood_compressed[task + "_" + value] = results_ood[value]
  
  task_cnt = Counter()
  for value in results_pos:
    for s in results_pos[value]:
      cnt = results_pos[value][s]
      for k in cnt:
        # print(s, results_pos[s])
        task_cnt[k] += cnt[k]
    results_pos_compressed[task + "_" + value] = task_cnt

... concordancia_de_sujeito_1_adotar
... concordancia_de_sujeito_1_afirmar
... concordancia_de_sujeito_1_comentar
... concordancia_de_sujeito_1_estar
... concordancia_de_sujeito_1_justificar
... concordancia_de_sujeito_1_ser
... concordancia_de_sujeito_2_carregar
... concordancia_de_sujeito_2_contribuir
... concordancia_de_sujeito_2_falar
... concordancia_de_sujeito_2_fazer
... concordância nominal_1a
... concordância nominal_1b
... concordância nominal_1c
... concordância nominal_1d
... concordância nominal_1e
... concordância nominal_2a
... concordância nominal_2b
... concordância nominal_2c
... concordância verbal_1_0
... concordância verbal_1_1
... concordância verbal_1_2
... concordância verbal_1_3
... concordância verbal_2_0
... concordância verbal_2_1
... concordância verbal_2_2
... concordância verbal_2_3
... concordância verbal_3a
... concordância verbal_3b
... concordância verbal_3c
... concordância verbal_3d
... concordância verbal_4_0
... concordância verbal_4_1
... concord

In [ ]:
with open("/content/drive/MyDrive/Aline et al/results_cls" + str(use_cls) + "_" + model_short_name + ".csv", "w") as output_file:
  sep = "\t"
  output_file.write("template"+sep+"seed\tsentence" +sep+ "all_pos" +sep+ "token_str" +sep+ "token_score" +sep+"wrong_pos"+sep+"lexis_comment" + "\n")
  for key in outputs_by_sent_seed:
    for all_pos, token_str, score in outputs_by_sent_seed[key]:
      # print(key,all_pos, token_str, score)
      output_file.write(key +sep+ all_pos +sep+ token_str +sep+ str(score) + "\n")


In [ ]:
for task in results_pos_compressed:
  print(task)
  print("\t", results_pos_compressed[task])
  print("\tOOD",results_ood_compressed[task])

concordancia_de_sujeito_1_adotar_Indicativo pretérito perfeito simples_sg
	 Counter({'PRO+Pes:A3fs': 6, 'PRO+Dem:fs': 6, 'CONJ': 6, 'ADV': 6, 'V:P3s': 5, 'V:Y2s': 4, 'N:ms': 4, 'PRO+Pes:A3ms': 4, 'PRO+Pes:A3fp': 4, 'N:mp': 4, 'N:fs': 3, 'PRO+Pes:N3fs': 3, 'PREP': 3, 'DET+Art+Def:fs': 3, 'PRO+Pes:R3ms': 3, 'PRO+Pes:R3fp': 3, 'PRO+Pes:R3mp': 3, 'PRO+Pes:R3fs': 3, 'PRO+Dem:ms': 3, 'DET+Art+Def:ms': 3, 'PRO+Dem:fp': 2, 'DET+Art+Def:fp': 2, 'PRO+Pes:N3fp': 2, 'V:P2s': 2, 'N:fp': 2, 'PRO+Pes:A3mp': 2, 'PRO+Dem:mp': 2, 'DET+Art+Def:mp': 2, 'V:S3s': 1, 'V:S1s': 1, 'V:Y3s': 1, 'PRO+Pes:N3ms': 1})
	OOD set()
concordancia_de_sujeito_1_afirmar_Indicativo pretérito perfeito simples_sg
	 Counter({'CONJ': 8, 'N:ms': 8, 'ADV': 6, 'V:P3s': 6, 'V:S3s': 5, 'V:S1s': 5, 'PRO+Pes:A3ms': 5, 'V:Y3s': 5, 'PRO+Pes:N3ms': 5, 'PRO+Pes:A3fs': 5, 'N:fs': 4, 'PRO+Pes:N3fs': 4, 'V:Y2s': 4, 'N+Pr:ms': 4, 'punct': 3, 'PRO+Pes:A3mp': 1, 'N:mp': 1, 'V:S2s': 1, 'PRO+Pes:N3mp': 1, 'N+Pr:fs': 1, 'OOD': 1, 'PRO+Dem:fs': 1, '

#old

In [ ]:
nlp_fill = pipeline('fill-mask',model="neuralmind/bert-large-portuguese-cased", use_fast=True, top_k=10)

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
%%time
use_cls = False
use_sep = False

results_pos_compressed = {}
results_ood_compressed = {}
for task in TASKS:
  print("...",task)
  results_scores, results_pos, results_ood = process(seeds=TASKS[task][0], sentences=TASKS[task][1], use_cls=use_cls, use_sep=use_sep, nlp_fill=nlp_fill, pos_dict=pos_dict, verbose = False)
  for value in results_ood:
    results_ood_compressed[task + "_" + value] = results_ood[value]
  
  task_cnt = Counter()
  for value in results_pos:
    for s in results_pos[value]:
      cnt = results_pos[value][s]
      for k in cnt:
        # print(s, results_pos[s])
        task_cnt[k] += cnt[k]
    results_pos_compressed[task + "_" + value] = task_cnt

... concordancia_de_sujeito_1_adotar
... concordancia_de_sujeito_1_afirmar
... concordancia_de_sujeito_1_comentar
... concordancia_de_sujeito_1_estar
... concordancia_de_sujeito_1_justificar
... concordancia_de_sujeito_1_ser
... concordancia_de_sujeito_2_carregar
... concordancia_de_sujeito_2_contribuir
... concordancia_de_sujeito_2_falar
... concordancia_de_sujeito_2_fazer
... concordância nominal_1a
... concordância nominal_1b
... concordância nominal_1c
... concordância nominal_1d
... concordância nominal_1e
... concordância nominal_2a
... concordância nominal_2b
... concordância nominal_2c
... concordância verbal_1_0
... concordância verbal_1_1
... concordância verbal_1_2
... concordância verbal_1_3
... concordância verbal_2_0
... concordância verbal_2_1
... concordância verbal_2_2
... concordância verbal_2_3
... concordância verbal_3a
... concordância verbal_3b
... concordância verbal_3c
... concordância verbal_3d
... concordância verbal_4_0
... concordância verbal_4_1
... concord

In [ ]:
for task in results_pos_compressed:
  print(task)
  print("\t", results_pos_compressed[task])
  print("\tOOD",results_ood_compressed[task])

concordancia_de_sujeito_1_adotar_Indicativo pretérito perfeito simples_sg
	 Counter({'ADV': 8, 'PRO+Pes:A3fs': 6, 'N:fs': 5, 'PRO+Dem:fs': 5, 'N:ms': 5, 'PREPXPRO+Pes:O3fs': 4, 'V:P3s': 3, 'V:Y2s': 3, 'PRO+Pes:N3fs': 3, 'CONJ': 3, 'PREP': 3, 'DET+Art+Def:fs': 3, 'PRO+Pes:N2ms': 2, 'PRO+Pes:N2fs': 2, 'PRO+Pes:R3fs': 1, 'PRO+Pes:R3ms': 1, 'PRO+Pes:R3mp': 1, 'PRO+Pes:R3fp': 1, 'V:S1s': 1, 'PRO+Pes:N3ms': 1, 'V:S3s': 1, 'V:Y3s': 1, 'PRO+Pes:A3ms': 1, 'PRO+Dem:ms': 1, 'A:ms': 1, 'A:fs': 1, 'N+Pr:fs': 1, 'N+Pr:ms': 1, 'PRO+Pes:N1fp': 1, 'PRO+Pes:N1mp': 1, 'N:mp': 1, 'PRO+Pes:N1fs': 1, 'PRO+Pes:N1ms': 1})
	OOD set()
concordancia_de_sujeito_1_afirmar_Indicativo pretérito perfeito simples_sg
	 Counter({'ADV': 12, 'N:ms': 11, 'CONJ': 10, 'N+Pr:ms': 6, 'N+Pr:fs': 4, 'PRO+Pes:N2ms': 3, 'PRO+Pes:N2fs': 3, 'PRO+Ind:ms': 3, 'N:fs': 3, 'PRO+Rel:ms': 2, 'V:P1s': 2, 'PRO+Pes:N1fs': 2, 'PRO+Pes:N1ms': 2, 'V:P3s': 2, 'V:Y2s': 2, 'N+Pr:fp': 2, 'N+Pr:mp': 2, 'PRO+Ind:fs': 1, 'PRO+Tra:3ms': 1, 'PRO+Tra:3fs':